In [3]:
import cv2
from PIL import Image
import glob, os
import json

In [8]:
# Need to convert the Cross-Recessed-Screw data from individual xml labels into a combined coco Json format
# Run that script seperately if the conversion is needed

In [4]:
def norm2abs(bbx_coords, image_shape):
    x, y, w, h, = bbx_coords

    # get x and y
    x_reg = int(x * image_shape[1])
    y_reg = int(y * image_shape[0])
    w_reg = int(w * image_shape[1])
    h_reg = int(h * image_shape[0])

    bbx_coords_reg = [x_reg, y_reg, w_reg, h_reg]
    return bbx_coords_reg

In [2]:
# Check image dimensions, bounding box coordinates and labels
# We will compare the normalised and absolute coordinates, we assume the two json files are keyed identically
root = 'C:/Users/drago/Documents/gitrepos/Cross-Recessed-Screw_Deep-Learning-Datasets/'
directory = root + 'Training_Images/'
check_shape = False

#abs_labels_dir = root + 'Completed_Labels.json'
abs_labels_dir = root + 'Training_Annotations/coco.json'

# Open the json file containing the absolute labels
with open(abs_labels_dir) as abs_labels_json:
    abs_labels_dict = json.load(abs_labels_json)

# Iterate through each image in abs_labels, extracting filename and id
for file in [{'name': image['file_name'].split('/')[-1], 'id': image['id']} for image in abs_labels_dict['images'][:]]:
    if file['name'].endswith('.png') or file['name'].endswith('.jpg'):
        # Read in the image
        dataset_img = cv2.imread(directory + file['name'])
        
        # Extract the bounding boxes for each screw in the image
        label_ids = []
        for index, abs_label in enumerate(abs_labels_dict['annotations'][:]):
            # Find the annotations that match the image
            # I want to know which images have annotations of area zero
            if abs_label['image_id'] == file['id']:

                # Read in absolute bounding box coordinates
                coordinates = abs_label['bbox']

                #Display image with bounding box
                start_point = (int(coordinates[0]), int(coordinates[1]))
                end_point = (int(coordinates[0] + coordinates[2]), int(coordinates[1] + coordinates[3]))
                
                print(start_point, end_point)
                print('\n')

                if abs_label['category_id'] == 2:
                    colour = (0, 0, 255)
                if abs_label['category_id'] == 4:
                    colour = (255, 0, 0)
                # draw the rectangle
                cv2.rectangle(dataset_img, start_point, end_point, color=colour, thickness= 1, lineType=cv2.LINE_8)

                print("Image details: {}".format(file))
                print("Label details: {}".format(abs_label))

        print("Zero area bounding box found in image {}".format(file['name']))
        # display the output
        print("Dimensions {}: {}".format(file['name'], dataset_img.shape))
        cv2.imshow('imageRectangle', dataset_img)
        if cv2.waitKey(0) == 27: break
        else: continue
    else:
        continue

cv2.destroyAllWindows()

# # Check Github dataset iamge dimensions
# directory = 'C:/Users/drago/Documents/gitrepos/Cross-Recessed-Screw_Deep-Learning-Datasets/Training_Images/'
# for filename in os.listdir(directory):
#     if filename.endswith('.jpg'):
#         dataset_img = cv2.imread(directory + filename)
        
#         print("Dimensions {}: {}", (filename, dataset_img.shape))
#         continue
#     else:
#         continue

# directory = 'C:/Users/drago/Documents/gitrepos/Cross-Recessed-Screw_Deep-Learning-Datasets/Training_Images/'

(1003, 1275) (1418, 1628)


Image details: {'name': '000001.jpg', 'id': 1}
Label details: {'id': 1, 'image_id': 1, 'category_id': 2, 'segmentation': [[1003, 1275, 1003, 1628, 1418, 1628, 1418, 1275]], 'area': 146495, 'bbox': [1003, 1275, 415, 353], 'iscrowd': 0}
Zero area bounding box found in image 000001.jpg
Dimensions 000001.jpg: (3036, 4048, 3)
(1346, 391) (1545, 606)


Image details: {'name': '000002.jpg', 'id': 2}
Label details: {'id': 2, 'image_id': 2, 'category_id': 2, 'segmentation': [[1346, 391, 1346, 606, 1545, 606, 1545, 391]], 'area': 42785, 'bbox': [1346, 391, 199, 215], 'iscrowd': 0}
(1324, 3299) (1506, 3442)


Image details: {'name': '000002.jpg', 'id': 2}
Label details: {'id': 3, 'image_id': 2, 'category_id': 2, 'segmentation': [[1324, 3299, 1324, 3442, 1506, 3442, 1506, 3299]], 'area': 26026, 'bbox': [1324, 3299, 182, 143], 'iscrowd': 0}
Zero area bounding box found in image 000002.jpg
Dimensions 000002.jpg: (4048, 3036, 3)
(550, 1966) (710, 2121)


Image details: {'n

In [6]:
# Convert git dataset to 480x640 dimensions
size = 480, 640

for infile in glob.glob(directory + "*.jpg"):
    file, ext = os.path.splitext(infile)
    file_name = os.path.basename(file)
    with Image.open(infile) as im:
        # When the image is in portrait mode, scale by flipped dimensions
        if im.size[0] > im.size[1]:
            im.thumbnail((640, 480))
        else:
            im.thumbnail((480, 640))
        
        im.save(directory + 'thumbnail/' + file_name + '.jpg', "JPEG")

In [ ]:

# Check those dimensions match all images within their respective dataset
# NOTE: There are some rotated 90 deg
# Now, think of some creative way to handle the difference between the image sizes,
#  the new cameras image size and the size of model needed to take these as inputs?
# Do I need to scale down the size of the images for prediction? How would I make use of the predictions?
#  Would scaling them actually work?